In [ ]:
import random
import numpy as np
import sys
from random import randint
from data_load import *
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
a=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_x_new.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")
%load_ext Cython

In [ ]:
%%cython
import numpy as np

cimport numpy as np
from random import randint
import sys
import cython

import math
from libc.stdlib cimport malloc, free

from libc.math cimport exp
from libc.math cimport log
from libc.math cimport sqrt
from libc.string cimport memset
import random
# scipy <= 0.15

import scipy.linalg.blas as fblas
ctypedef np.float32_t REAL_t
cdef int ONE = 1


REAL = np.float32
cdef extern from "/Users/mayk/working/figer/baseline/PLE/Model/warp/voidptr.h":
    void* PyCObject_AsVoidPtr(object obj)
DEF MAX_SENTENCE_LEN = 10000
ctypedef void (*scopy_ptr) (const int *N, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef void (*saxpy_ptr) (const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef float (*sdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*dsdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*snrm2_ptr) (const int *N, const float *X, const int *incX) nogil
ctypedef void (*sscal_ptr) (const int *N, const float *alpha, const float *X, const int *incX) nogil


cdef scopy_ptr scopy = <scopy_ptr>PyCObject_AsVoidPtr(fblas.scopy._cpointer)  # y = x
cdef saxpy_ptr saxpy=<saxpy_ptr>PyCObject_AsVoidPtr(fblas.saxpy._cpointer)  # y += alpha * x
cdef sdot_ptr sdot=<sdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # float = dot(x, y)
cdef dsdot_ptr dsdot=<dsdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # double = dot(x, y)
cdef snrm2_ptr snrm2=<snrm2_ptr>PyCObject_AsVoidPtr(fblas.snrm2._cpointer)  # sqrt(x^2)
cdef sscal_ptr sscal=<sscal_ptr>PyCObject_AsVoidPtr(fblas.sscal._cpointer) # x = alpha * x
DEF EXP_TABLE_SIZE = 10000
DEF MAX_EXP = 50

cdef REAL_t[EXP_TABLE_SIZE] EXP_TABLE
cdef REAL_t[EXP_TABLE_SIZE] LOG_TABLE

cdef REAL_t ONEF = <REAL_t>1.0

# for when fblas.sdot returns a double
cdef REAL_t our_dot_double(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>dsdot(N, X, incX, Y, incY)

# for when fblas.sdot returns a float
cdef REAL_t our_dot_float(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>sdot(N, X, incX, Y, incY)

# for when no blas available
cdef REAL_t our_dot_noblas(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    # not a true full dot()-implementation: just enough for our cases
    cdef int i
    cdef REAL_t a
    a = <REAL_t>0.0
    for i from 0 <= i < 50 by 1:
        a += X[i] * Y[i]
    return a

# for when no blas available
cdef void our_saxpy_noblas(const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil:
    cdef int i
    for i from 0 <= i < N[0] by 1:
        Y[i * (incY[0])] = (alpha[0]) * X[i * (incX[0])] + Y[i * (incY[0])]
cdef REAL_t cvdot(vec1,vec2,size):
    cdef int csize = size
    f= dsdot(&csize,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
def csaxpy(vec1,vec2,alpha,size):
    cdef int csize = size
    cdef float calpha = alpha
    f= our_saxpy_noblas(&csize,&calpha,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
cdef REAL_t crank(int k):
    cdef REAL_t loss = 0.
    cdef int i = 1
    for i in xrange(1,k+1):
        loss += ONEF/i
    return loss
cdef REAL_t vsum(REAL_t *vec,int *size):
    cdef int i
    cdef REAL_t product
    product = <REAL_t>0.0
    for i from 0 <= i < size[0] by 1:
        product += vec[i] * vec[i]
    return np.sqrt(product)
def cnorm(vec):
    cdef int size
    size  = len(vec)
    return vsum(<REAL_t *>(np.PyArray_DATA(vec)),&size)
def init():
    for i in range(EXP_TABLE_SIZE):
        EXP_TABLE[i] = <REAL_t>exp((i / <REAL_t>EXP_TABLE_SIZE * 2 - 1) * MAX_EXP)
        EXP_TABLE[i] = <REAL_t>(EXP_TABLE[i] / (EXP_TABLE[i] + 1))
#init()

cdef void divide(REAL_t *vec, const float *alpha, const int *size):
    cdef int i
    cdef REAL_t product
    for i from 0 <= i < size[0] by 1:
        vec[i] = vec[i]/alpha[0]
def cdivide(vec,alpha):
    cdef int size
    size  = len(vec)
    cdef float r = alpha
    divide(<REAL_t *>(np.PyArray_DATA(vec)),&r,&size)




def pure_ctrain(A,B,Ys,pcnts,Xs,indices,size,lr,label_size,max_it =10):
    cdef REAL_t error
    cdef REAL_t *cA = <REAL_t *>(np.PyArray_DATA(A))
    cdef REAL_t *cB = <REAL_t *>(np.PyArray_DATA(B))
    cdef int rows = label_size
    cdef REAL_t *dB = <REAL_t *>malloc(size*rows * sizeof(REAL_t))
    cdef REAL_t *dA =  <REAL_t *>malloc(size * sizeof(REAL_t))
    cdef REAL_t *x =  <REAL_t *>malloc(size * sizeof(REAL_t))
    cdef int *f_idx = <int *>(np.PyArray_DATA(indices)) 
    cdef int *labels = <int *>(np.PyArray_DATA(Ys))
    cdef int *pos_cnts = <int *>(np.PyArray_DATA(pcnts))
    cdef int *features =  <int *>(np.PyArray_DATA(Xs))
    
    cdef int cSize = size
    for it in xrange(1,max_it+1):
        error = 0.
        for i,inst in enumerate(pcnts):
            error+=cwarp_gradient(cA,cB,dB,dA,x,&features[f_idx[i]],f_idx[i+1]-f_idx[i],&labels[i*label_size],pos_cnts[i],label_size,cSize,lr)
        
            if i % 1000 ==0:
                sys.stdout.write("\rIteration %d " % (it)+ "trained {0:.0f}%".format(float(i)*100/len(pcnts))+" Loss:{0:.2f}".format(error))
                sys.stdout.flush()
        sys.stdout.write("\n")
    free(dA)
    free(dB)
    free(x)
    return None
cdef REAL_t cwarp_gradient(REAL_t *cA,REAL_t *cB,REAL_t *dB,REAL_t *dA,
                                       REAL_t *x,int *features,int fcnt, int *label,int pos_cnt,int label_cnt,
                                       int cSize,float clr):
    memset(dB,0, cSize *label_cnt* cython.sizeof(REAL_t)) 
    memset(dA, 0, cSize * cython.sizeof(REAL_t))
    memset(x, 0, cSize * cython.sizeof(REAL_t))
    for i in range(fcnt):
        our_saxpy_noblas(&cSize,&ONEF,&cA[features[i]*cSize],&ONE,x,&ONE)
    random.seed(1)
    cdef REAL_t error = 0.0
    cdef REAL_t s1,s2
    cdef float L,negL
    cdef int N,n_sample 
    cdef int neg_num = label_cnt-pos_cnt
    cdef REAL_t norm = <REAL_t>0.0
    
    

    for pos in range(pos_cnt):
        l = label[pos]
        s1 = dsdot(&cSize,&cB[l*cSize],&ONE,x,&ONE)
        N=1
        n_sample  = -1
        for k in xrange(neg_num):
            nl = label[pos_cnt+randint(0,neg_num-1)]
            s2 = dsdot(&cSize,&cB[nl*cSize],&ONE,x,&ONE)
            if s1 - s2<1:
                n_sample = nl
                N = k+1
                break
        if n_sample!=-1:
            L = crank(neg_num/N)
            negL = -L
            error += (1+s2-s1)*L            
            our_saxpy_noblas(&cSize,&L,&cB[l*cSize],&ONE,dA,&ONE)
            our_saxpy_noblas(&cSize,&negL,&cB[n_sample*cSize],&ONE,dA,&ONE)

            our_saxpy_noblas(&cSize,&L,x,&ONE,&dB[l*cSize],&ONE)
            our_saxpy_noblas(&cSize,&negL,x,&ONE,&dB[n_sample*cSize],&ONE)
            
    for i in range(fcnt):
        f = features[i]
    
        our_saxpy_noblas(&cSize,&clr,dA,&ONE,&cA[f*cSize],&ONE)
        norm = <REAL_t>0.0

        for m in range(cSize):
            #print cA[f*cSize+m]
            norm+= cA[f*cSize+m]*cA[f*cSize+m]
        
        norm = sqrt(norm)
     
        if norm >1:
            divide(&cA[f*cSize],&norm,&cSize)

    for i in xrange(label_cnt):
        our_saxpy_noblas(&cSize,&clr,&dB[i*cSize],&ONE,&cB[i*cSize],&ONE)
        norm = <REAL_t>0.0
         
        for m in range(cSize):
            norm+= cB[i*cSize+m]*cB[i*cSize+m]
        norm = sqrt(norm)

         #norm =  vsum(&cB[i*cSize],&cSize)
        if norm >1:
            divide(&cB[i*cSize],&norm,&cSize)


    return error


def save_to_text(matrix,output):
    shape = matrix.shape
    with open(output,'wb') as out:
        out.write("%d %d\n" % (shape))
        for row in matrix:
            x = " ".join(map(lambda x:"{0:.5}".format(x),row))
            out.write(x+"\n")



In [ ]:
size=50
np.random.seed(1)
A= np.random.rand(len(a.feature2id),size).astype(np.float32)
for i in xrange(len(A)):
    norm =  cnorm(A[i])
    if norm >1:
        cdivide(A[i],norm)
B= np.random.rand(len(a.label2id),size).astype(np.float32)
for i in xrange(len(B)):
    norm =  cnorm(B[i])
    if norm >1:
        cdivide(B[i],norm)
insts  = a.data[:2500]
labels = np.array([inst.sparse_labels+inst.negative_labels for inst in insts],dtype=np.int32)
pos_cnts = np.array([len(inst.sparse_labels)for inst in insts],dtype=np.int32)
feats = []
for inst in insts:
    feats.extend(inst.features)
features = np.array(feats,dtype=np.int32)
indices = []
cnt = 0
for inst in insts:
    indices.append(cnt)
    cnt += len(inst.features)
indices.append(cnt)
indxes = np.array(indices,dtype=np.int32)
pure_ctrain(A,B,labels,pos_cnts,features,indxes,50,0.01,len(a.label2id),max_it=2)
#save_to_text(A,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_A.txt')
#save_to_text(B,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_B.txt')